In [ ]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol

using StaticArrays
using Rotations
using Distances
using LinearAlgebra
using CairoMakie
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [43]:
# Consecutive assembly
R0 = RotMatrix(@SMatrix[1.000000  0.000000  0.000000; 0.000000  1.000000  0.000000; 0.000000  0.000000  1.000000])
T0 = @SVector[0.00000, 0.00000, 0.00000]

R1 = RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000])
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000])
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = RotMatrix(@SMatrix[0.992567  0.121696  0.000000; -0.121696  0.992567  0.000000; 0.000000  0.000000  1.000000])
T3 = @SVector[-19.48193, 22.01644, -46.53000]

template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["template_centers"]
consecutive = Vector{Float64}([])
for (R,T) in zip([log(R1), log(R2), log(R3)], [T1, T2, T3])
    consecutive = [consecutive; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end
points = [Vector{Float64}(e) for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(experimental_state, template_centers)...))]
MorphoMolNotebooks.configuration_to_poly(points, radii, colors, "assets/3su_test/exp_1")

# Two top one bottom
R1 = RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000 1.000000])
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000 1.000000])
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = RotMatrix(@SMatrix[0.52145615 0.85327808 0.000000; -0.85327808 0.52145615 -0.000000; 0.000000 0.000000 1.000000])
T3 = @SVector[-63.89221, 227.07555, -26.79]

R4 = RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384 0.803441 0.000000; 0.000000 0.000000 1.000000])
T4 = @SVector[-67.99970, 135.02619, -25.38000]

two_top_one_bottom = Vector{Float64}([])
for (R,T) in zip([log(R1), log(R2), log(R4)], [T1, T2, T4])
    two_top_one_bottom = [two_top_one_bottom; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end
points = [Vector{Float64}(e) for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(experimental_state, template_centers)...))]
MorphoMolNotebooks.configuration_to_poly(points, radii, colors, "assets/3su_test/exp_2")

# Two bottom one top
R1 = RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000 1.000000])
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000 1.000000])
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = RotMatrix(@SMatrix[0.52145615 0.85327808 0.000000; -0.85327808 0.52145615 -0.000000; 0.000000 0.000000 1.000000])
T3 = @SVector[-63.89221, 227.07555, -26.79]

R4 = RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384 0.803441 0.000000; 0.000000 0.000000 1.000000])
T4 = @SVector[-67.99970, 135.02619, -25.38000]

two_bottom_one_top = Vector{Float64}([])
for (R,T) in zip([log(R1), log(R3), log(R4)], [T1, T3, T4])
    two_bottom_one_top = [two_bottom_one_top; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end
points = [Vector{Float64}(e) for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(experimental_state, template_centers)...))]
MorphoMolNotebooks.configuration_to_poly(points, radii, colors, "assets/3su_test/exp_3")

In [ ]:
folder = "../../Data/hpc_out/rwm_ma_3_6r7m/"
mol_type = "6r7m"
#folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end
            
            id = parse(Int, split(file, separator)[id_index])
            template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]

            mindex = argmin(output["Es"])
            simulated_assembly_state = output["states"][mindex];

            theta_string = ""
            permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
            for (name, experimental_state) in [("consecutive", consecutive), ("two_top_one_bottom", two_top_one_bottom), ("two_bottom_one_top", two_bottom_one_top)]
                dists = Vector{Float64}([])
                for perm in permutations
                    push!(dists, MorphoMol.Utilities.sum_of_permutation(template_centers, template_centers, simulated_assembly_state, experimental_state, [1, 2, 3], perm))
                end
                theta_string = theta_string * "$(name): $(minimum(dists)) |"
            end
            string = "$(id): $(input["T"]) | $(input["overlap_slope"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta_string)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")
for elem in evaluation_strings
    println(elem)
end

Minimal configuration id: 87 | Average acceptance rate: 0.6287543772799629
1: 5.0 | 0.95 | 0.4085963 | 13144.112295135672, 13027.902624831155, 13111.93144844484 | consecutive: 270.4103123137863 |two_top_one_bottom: 274.91461176526803 |two_bottom_one_top: 274.91458439277903 |
2: 5.0 | 0.95 | 0.6701133 | 13144.112295549847, 13016.611493917204, 13016.611493917204 | consecutive: 439.51833361783946 |two_top_one_bottom: 444.0226330693211 |two_bottom_one_top: 444.02260569683216 |
3: 4.0 | 1.0 | 0.6872012 | 13144.11229548463, 13073.918459747012, 13077.773074498962 | consecutive: 461.1931204150371 |two_top_one_bottom: 465.6974198665187 |two_bottom_one_top: 465.6973924940298 |
4: 5.0 | 0.95 | 0.76721317 | 13144.11229571789, 13076.351821920165, 13114.503348301223 | consecutive: 137.08279915438857 |two_top_one_bottom: 141.5870986058703 |two_bottom_one_top: 141.58707123338127 |
5: 4.0 | 1.0 | 0.55513513 | 13144.112295375311, 13086.165889485652, 13144.11229530796 | consecutive: 466.8502098702302 |tw

In [52]:
#file = "../SolSim.jl_dev/io/hpc/all_data/three_monomers/protor_oj_0_0_os_0_85/6r7m/5/75_6r7m_protor_oj_0_0_os_0_85.jld2"
file = "../../Data/hpc_out/rwm_ma_3_6r7m/156_rwm_ma_3_6r7m.jld2"
@load file input output
n_mol = input["n_mol"]
n_atoms_per_mol = length(input["template_radii"])

mindex = argmin(output["Es"])
simulated_assembly_state = output["states"][mindex];

points = [Vector{Float64}(e) for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(simulated_assembly_state, input["template_centers"])...))]
radii = [input["template_radii"]; input["template_radii"]; input["template_radii"]];
colors = [MorphoMolNotebooks.STANDARD_COLORS[j] for j in 1:n_mol for i in 1:n_atoms_per_mol]
MorphoMolNotebooks.configuration_to_poly(points, radii, colors, "assets/3su_test/sa")

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for (name, experimental_state) in [("consecutive", consecutive), ("two_top_one_bottom", two_top_one_bottom), ("two_bottom_one_top", two_bottom_one_top), ("simulated_assembly_state", simulated_assembly_state)]
    dists = Vector{Float64}([])
    for perm in permutations
        d = MorphoMol.Utilities.sum_of_permutation(template_centers, template_centers, simulated_assembly_state, experimental_state, [1, 2, 3], perm)
        println("$(perm) | $(d)")
        push!(dists, d)
    end
    println("$(name) | $(minimum(dists))\n")
end

42.97171653296904, 92.08973481384777, 74.69139281774989
[1, 2, 3] | 69.91761472152223
21.601927563493025, 92.08973481384777, 96.06118178722589
[1, 3, 2] | 69.91761472152223
42.97171653296904, 70.72005462595087, 96.06107300564682
[2, 1, 3] | 69.91761472152224
42.97160775138994, 70.72005462595087, 96.06118178722589
[2, 3, 1] | 69.91761472152223
21.601927563493025, 92.08984359542688, 96.06107300564682
[3, 1, 2] | 69.91761472152224
42.97160775138994, 92.08984359542688, 74.69139281774989
[3, 2, 1] | 69.91761472152224
consecutive | 69.91761472152223

42.97171653296904, 90.3751323624556, 89.91889362358734
[1, 2, 3] | 74.421914173004
36.829428369330465, 90.3751323624556, 96.06118178722589
[1, 3, 2] | 74.42191417300398
42.97171653296904, 85.94755543178832, 94.34647055425461
[2, 1, 3] | 74.42191417300398
41.25700529999776, 85.94755543178832, 96.06118178722589
[2, 3, 1] | 74.42191417300398
36.829428369330465, 92.08984359542688, 94.34647055425461
[3, 1, 2] | 74.42191417300398
41.25700529999776, 92

In [50]:
42.97171653296904 + 70.72005462595087 + 96.06107300564682

209.75284416456674